## E91 Protocol   
This protocol is based on the property of entanglement of photons. Alice and Bob each receive one qubit each, of a pre-decided bell state. We take the bell state 1/sqrt(2)(|00⟩+|11⟩) for the simulation. Alice and Bob prepare a list recording the information on the basis they use for each measurement. Denoting the rectilinear basis as '0' and the diagonal basis as '1'. They then perform the measurement and store the results in another list. After the measurement, they send their basis choices' list to each other and note the indices when they measured in the same basis. Because of entanglement, their measurement results corresponding to these indices would be same and they derive a key by making a key bit string out of them 

In [1]:
from qiskit import *
from qiskit.tools.visualization import circuit_drawer, plot_histogram
import numpy as np
from qiskit.extensions import Initialize
import random

In [2]:
qr = QuantumRegister(2)
cr = ClassicalRegister(2)

#creating entangled state
circuit = QuantumCircuit(qr, cr)
circuit.h(qr[0])
circuit.cx(qr[0],qr[1])

n = 8    
choice_alice = [0] * n
choice_bob = [0] * n

result_alice = [0] * n
result_bob = [0] * n

for i in range(n) : 
    choice = [random.randint(0,1),random.randint(0,1)]
    choice_alice[i] = choice[0]
    choice_bob[i] = choice[1]
    measurement_circuit = QuantumCircuit(qr,cr)
    if choice[0] == 1 : 
        measurement_circuit.h(qr[0])
    if choice[1] == 1 : 
        measurement_circuit.h(qr[1])
    final_circ = circuit + measurement_circuit
    final_circ.measure(qr,cr)
    backend = Aer.get_backend('qasm_simulator')
    job = execute(final_circ, backend = backend, shots =1)
    print(final_circ)
    
    
    results = job.result()
    answer = results.get_counts(final_circ)
    output = list(answer.keys())[list(answer.values()).index(1)]
    #print("Measurement Result:",output)
    #print(type(output))
    output_list = list(map(int,output))
    #print(output_list)
    
    
    result_alice[i] = output_list[1]
    result_bob[i] = output_list[0]
    #print("--------------------------------------------")
    #print(answer)
    
print("A :",choice_alice)
print("B :",choice_bob)

print("A':",result_alice)
print("B':",result_bob)
key_alice = []
key_bob = []

for i in range(n) : 
    if choice_alice[i] == choice_bob[i] : 
        key_alice.append(result_alice[i])
        key_bob.append(result_bob[i])


if key_alice == key_bob : 
    print("key shared successfully:")
    print(key_alice)
    


      ┌───┐     ┌─┐   
q0_0: ┤ H ├──■──┤M├───
      └───┘┌─┴─┐└╥┘┌─┐
q0_1: ─────┤ X ├─╫─┤M├
           └───┘ ║ └╥┘
c0_0: ═══════════╩══╬═
                    ║ 
c0_1: ══════════════╩═
                      
      ┌───┐     ┌───┐┌─┐   
q0_0: ┤ H ├──■──┤ H ├┤M├───
      └───┘┌─┴─┐├───┤└╥┘┌─┐
q0_1: ─────┤ X ├┤ H ├─╫─┤M├
           └───┘└───┘ ║ └╥┘
c0_0: ════════════════╩══╬═
                         ║ 
c0_1: ═══════════════════╩═
                           
      ┌───┐     ┌───┐┌─┐   
q0_0: ┤ H ├──■──┤ H ├┤M├───
      └───┘┌─┴─┐├───┤└╥┘┌─┐
q0_1: ─────┤ X ├┤ H ├─╫─┤M├
           └───┘└───┘ ║ └╥┘
c0_0: ════════════════╩══╬═
                         ║ 
c0_1: ═══════════════════╩═
                           
      ┌───┐     ┌─┐   
q0_0: ┤ H ├──■──┤M├───
      └───┘┌─┴─┐└╥┘┌─┐
q0_1: ─────┤ X ├─╫─┤M├
           └───┘ ║ └╥┘
c0_0: ═══════════╩══╬═
                    ║ 
c0_1: ══════════════╩═
                      
      ┌───┐     ┌───┐┌─┐   
q0_0: ┤ H ├──■──┤ H ├┤M├───
      └───┘┌─┴─┐├───┤└╥┘┌─

The first part of these circuits is creating the bell state and second part shows which basis Alice and Bob are measuring in. Here, A & B represent the measurement basis chosen by each and A' and B' represent the measurement results. We can see that the key was derived from the indices where the measurement was performed in the same basis

## Eavesdropping in E91  
One drawback of this protocol is that an eavesdropper can measure the bell state being sent and send an un-entangled state to Alice and Bob such that Eve's measurement match with Alice and Bob's. For example if Eve measures |0⟩|0⟩ then she would send the qubit |0⟩ to both Alice and Bob. A representation of such a case has been simulated below 

In [3]:
n = 12

qr = QuantumRegister(2)
cr = ClassicalRegister(4)

#creating entangled state
circuit = QuantumCircuit(qr, cr)
circuit.h(qr[0])
circuit.cx(qr[0],qr[1])
circuit.barrier()

send_alice = [0] * n
send_bob = [0] * n
choice_alice = [0]*n
choice_bob = [0]*n

result_alice = [0]*n
result_bob = [0]*n

for i in range(n) :         
    eves_circ = QuantumCircuit(qr,cr)
    eves_circ.measure([qr[0],qr[1]],[cr[2],cr[3]])
    eavesdropping = circuit + eves_circ 
    
    backend = Aer.get_backend('qasm_simulator')
    job = execute(eavesdropping, backend = backend, shots =1)
    results = job.result()
    answer = results.get_counts(eavesdropping)
    output = list(answer.keys())[list(answer.values()).index(1)]
    #print("Measurement Result:",output)
    #print(type(output))
    
    
    output_list = list(map(int,output))
    print("Measurement outcome that Eve receives     :", output_list)
    send_alice[i] = output_list[1]
    send_bob[i] = output_list[0]
    index = 2 * send_alice[i] + 1 * send_bob[i]

    fake_state = [0] * 4
    for j in range(4) : 
        if j == index : 
            fake_state[j] = 1


    #print("index",index)        
    print("Fake state that Eve sends to Alice and Bob:",fake_state)


    initializer = Initialize(fake_state)
    eavesdropping.append(initializer, [0,1])
    eavesdropping.barrier()
    
    
    #print(eavesdropping)
    
    print("The state", send_alice[i], "was sent to alice")
    print("The state", send_bob[i], "was sent to bob")
    
    
    choice = [random.randint(0,1),random.randint(0,1)]
    choice_alice[i] = choice[0]
    choice_bob[i] = choice[1]
    measurement_circuit = QuantumCircuit(qr,cr)
    if choice[0] == 1 : 
        measurement_circuit.h(qr[0])
    if choice[1] == 1 : 
        measurement_circuit.h(qr[1])
    
    final_circuit = eavesdropping + measurement_circuit
    final_circuit.measure([qr[0],qr[1]],[cr[0],cr[1]])
    #print(final_circuit)

    backend = Aer.get_backend('qasm_simulator')
    job = execute(final_circuit, backend = backend, shots =1)
    #print(final_circ)
    results = job.result()
    answer = results.get_counts(final_circuit)
    output = list(answer.keys())[list(answer.values()).index(1)]
    #print("Measurement Result:",output)
    #print(type(output))
    output_list = list(map(int,output))
    print("Alice measures:",output_list[1],"and Bob measures", output_list[0])
    result_alice[i] = output_list[1]
    result_bob[i] = output_list[0]
    print(final_circuit)
    print("-------------------------------------------------------------")
    
print("A :",choice_alice)
print("B :",choice_bob)

print("A':",result_alice)
print("B':",result_bob)
key_alice = []
key_bob = []

for i in range(n) : 
    if choice_alice[i] == choice_bob[i] : 
        key_alice.append(result_alice[i])
        key_bob.append(result_bob[i])


if key_alice == key_bob : 
    print("key shared successfully:")
    print(key_alice)
    
#final_circ.draw(output='m

Measurement outcome that Eve receives     : [0, 0, 0, 0]
Fake state that Eve sends to Alice and Bob: [1, 0, 0, 0]
The state 0 was sent to alice
The state 0 was sent to bob
Alice measures: 1 and Bob measures 1
      ┌───┐      ░ ┌─┐   ┌──────────────────────┐ ░ ┌───┐┌─┐   
q1_0: ┤ H ├──■───░─┤M├───┤0                     ├─░─┤ H ├┤M├───
      └───┘┌─┴─┐ ░ └╥┘┌─┐│  initialize(1,0,0,0) │ ░ ├───┤└╥┘┌─┐
q1_1: ─────┤ X ├─░──╫─┤M├┤1                     ├─░─┤ H ├─╫─┤M├
           └───┘ ░  ║ └╥┘└──────────────────────┘ ░ └───┘ ║ └╥┘
c1_0: ══════════════╬══╬══════════════════════════════════╩══╬═
                    ║  ║                                     ║ 
c1_1: ══════════════╬══╬═════════════════════════════════════╩═
                    ║  ║                                       
c1_2: ══════════════╩══╬═══════════════════════════════════════
                       ║                                       
c1_3: ═════════════════╩═══════════════════════════════════════
                       

* We can see how Eve creates a fake state matching with the result she receives and manipulates ALice and Bob to believe their communication is secure. They perform the protocol as before and do not realize the eavesdropping.
* To avoid this situation, ALice and Bob need to ensure that the qubits they receive is enatngled. They can do so by using three bases for measurement and by calculating the CHSH value. 